In [2]:
import pandas as pd
import numpy as np

In [58]:
df_train = pd.read_csv("./drive/MyDrive/Datasets/train.csv")
df_test = pd.read_csv("./drive/MyDrive/Datasets/test.csv")

In [133]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error

In [61]:
df_train.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [62]:
df_test.head(5)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [63]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [64]:
train_missing_value = df_train.isna().sum()

train_missing_value.to_csv('train missing value.csv')

In [65]:
x = pd.read_csv('train missing value.csv')
x[x['0'] > 0]

,Unnamed: 0,0
3,LotFrontage,259
6,Alley,1369
25,MasVnrType,872
26,MasVnrArea,8
30,BsmtQual,37
31,BsmtCond,37
32,BsmtExposure,38
33,BsmtFinType1,37
35,BsmtFinType2,38
42,Electrical,1


In [66]:
print(f'sum missing values: {df_train.isna().sum().sum().item()}')
print(f'duplicated values: {df_train.duplicated().sum().item()}')

sum missing values: 7829
duplicated values: 0


In [119]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   float64
 1   MSZoning       1459 non-null   object 
 2   LotFrontage    1459 non-null   float64
 3   LotArea        1459 non-null   float64
 4   Street         1459 non-null   object 
 5   LotShape       1459 non-null   object 
 6   LandContour    1459 non-null   object 
 7   Utilities      1459 non-null   object 
 8   LotConfig      1459 non-null   object 
 9   LandSlope      1459 non-null   object 
 10  Neighborhood   1459 non-null   object 
 11  Condition1     1459 non-null   object 
 12  Condition2     1459 non-null   object 
 13  BldgType       1459 non-null   object 
 14  HouseStyle     1459 non-null   object 
 15  OverallQual    1459 non-null   float64
 16  OverallCond    1459 non-null   float64
 17  YearBuilt      1459 non-null   float64
 18  YearRemo

In [68]:
test_missing_value = df_test.isna().sum()

test_missing_value.to_csv('test missing value.csv')

In [69]:
x = pd.read_csv('test missing value.csv')
x[x['0'] > 0]

,Unnamed: 0,0
2,MSZoning,4
3,LotFrontage,227
6,Alley,1352
9,Utilities,2
23,Exterior1st,1
24,Exterior2nd,1
25,MasVnrType,894
26,MasVnrArea,15
30,BsmtQual,44
31,BsmtCond,45


In [70]:
print(f'sum missing values: {df_test.isna().sum().sum().item()}')
print(f'duplicated values: {df_test.duplicated().sum().item()}')

sum missing values: 7878
duplicated values: 0


In [71]:
df_train.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
697,698,20,RL,57.0,6420,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Normal,123500
874,875,50,RM,52.0,5720,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2009,WD,Abnorml,66500
759,760,60,RL,65.0,12257,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,11,2007,WD,Normal,290000
510,511,20,RL,75.0,14559,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,Shed,2000,6,2009,WD,Normal,164900
210,211,30,RL,67.0,5604,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Normal,98000


In [72]:
df_train.drop(columns=['Id', 'Alley', 'PoolQC', 'Fence', 'MiscFeature'], inplace=True)

In [73]:
df_train.sample(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
197,75,RL,174.0,25419,Pave,Reg,Lvl,AllPub,Corner,Gtl,...,552,0,0,512,0,3,2006,WD,Abnorml,235000
970,50,RL,60.0,10800,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,12,2006,WD,Abnorml,135000
167,60,RL,86.0,10562,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,11,2007,New,Partial,325624
295,80,RL,37.0,7937,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,...,0,0,0,0,0,3,2006,WD,Normal,142500
1052,60,RL,100.0,9500,Pave,Reg,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,0,6,2007,WD,Normal,165000


In [74]:
numerical_cols = df_train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df_train.select_dtypes(include=['object']).columns


imputer_median = SimpleImputer(strategy='median')
df_train[numerical_cols] = imputer_median.fit_transform(df_train[numerical_cols])


imputer_most = SimpleImputer(strategy='most_frequent')
df_train[categorical_cols] = imputer_most.fit_transform(df_train[categorical_cols])

In [75]:
print(f'sum missing values: {df_train.isna().sum().sum().item()}')
print(f'duplicated values: {df_train.duplicated().sum().item()}')

sum missing values: 0
duplicated values: 0


In [76]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   float64
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   float64
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   float64
 16  OverallCond    1460 non-null   float64
 17  YearBuilt      1460 non-null   float64
 18  YearRemo

In [77]:
df_test.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
362,1823,30,C (all),72.0,9392,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2009,WD,Abnorml
410,1871,20,RL,78.0,11645,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,3,2009,WD,Normal
71,1532,30,RM,56.0,4485,Pave,Grvl,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,5,2010,WD,Normal
294,1755,85,RL,NaN,16500,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2009,WD,Normal
1372,2833,20,RL,78.0,11645,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2006,New,Partial


In [78]:
df_test.drop(columns=['Id', 'Alley', 'PoolQC', 'Fence', 'MiscFeature'], inplace=True)

In [79]:
df_test.sample(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1334,50,RM,51.0,6120,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,55,0,0,0,0,2,2006,WD,Normal
369,70,RM,57.0,8094,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,150,0,0,0,0,6,2009,WD,Normal
586,60,RL,73.0,8814,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,6,2008,WD,Normal
1142,180,RM,NaN,1533,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,97,0,0,0,0,0,7,2007,WD,Normal
1370,20,RL,64.0,7488,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,58,0,0,0,0,0,2,2006,WD,Abnorml


In [80]:
numerical_cols = df_test.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df_test.select_dtypes(include=['object']).columns


imputer_median = SimpleImputer(strategy='median')
df_test[numerical_cols] = imputer_median.fit_transform(df_test[numerical_cols])


imputer_most = SimpleImputer(strategy='most_frequent')
df_test[categorical_cols] = imputer_most.fit_transform(df_test[categorical_cols])

In [81]:
print(f'sum missing values: {df_test.isna().sum().sum().item()}')
print(f'duplicated values: {df_test.duplicated().sum().item()}')

sum missing values: 0
duplicated values: 0


In [82]:
df_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   float64
 1   MSZoning       1459 non-null   object 
 2   LotFrontage    1459 non-null   float64
 3   LotArea        1459 non-null   float64
 4   Street         1459 non-null   object 
 5   LotShape       1459 non-null   object 
 6   LandContour    1459 non-null   object 
 7   Utilities      1459 non-null   object 
 8   LotConfig      1459 non-null   object 
 9   LandSlope      1459 non-null   object 
 10  Neighborhood   1459 non-null   object 
 11  Condition1     1459 non-null   object 
 12  Condition2     1459 non-null   object 
 13  BldgType       1459 non-null   object 
 14  HouseStyle     1459 non-null   object 
 15  OverallQual    1459 non-null   float64
 16  OverallCond    1459 non-null   float64
 17  YearBuilt      1459 non-null   float64
 18  YearRemo

In [83]:
encoder = LabelEncoder()

df_train['MSZoning'] = encoder.fit_transform(df_train['MSZoning'])
df_train['Street'] = encoder.fit_transform(df_train['Street'])
df_train['LotShape'] = encoder.fit_transform(df_train['LotShape'])
df_train['LandContour'] = encoder.fit_transform(df_train['LandContour'])
df_train['Utilities'] = encoder.fit_transform(df_train['Utilities'])
df_train['LotConfig'] = encoder.fit_transform(df_train['LotConfig'])
df_train['LandSlope'] = encoder.fit_transform(df_train['LandSlope'])
df_train['Neighborhood'] = encoder.fit_transform(df_train['Neighborhood'])
df_train['Condition1'] = encoder.fit_transform(df_train['Condition1'])
df_train['Condition2'] = encoder.fit_transform(df_train['Condition2'])
df_train['BldgType'] = encoder.fit_transform(df_train['BldgType'])
df_train['HouseStyle'] = encoder.fit_transform(df_train['HouseStyle'])
df_train['RoofStyle'] = encoder.fit_transform(df_train['RoofStyle'])
df_train['RoofMatl'] = encoder.fit_transform(df_train['RoofMatl'])
df_train['Exterior1st'] = encoder.fit_transform(df_train['Exterior1st'])
df_train['Exterior2nd'] = encoder.fit_transform(df_train['Exterior2nd'])
df_train['MasVnrType'] = encoder.fit_transform(df_train['MasVnrType'])
df_train['ExterQual'] = encoder.fit_transform(df_train['ExterQual'])
df_train['ExterCond'] = encoder.fit_transform(df_train['ExterCond'])
df_train['Foundation'] = encoder.fit_transform(df_train['Foundation'])
df_train['BsmtQual'] = encoder.fit_transform(df_train['BsmtQual'])
df_train['BsmtCond'] = encoder.fit_transform(df_train['BsmtCond'])
df_train['BsmtExposure'] = encoder.fit_transform(df_train['BsmtExposure'])
df_train['BsmtFinType1'] = encoder.fit_transform(df_train['BsmtFinType1'])
df_train['BsmtFinType2'] = encoder.fit_transform(df_train['BsmtFinType2'])
df_train['Heating'] = encoder.fit_transform(df_train['Heating'])
df_train['HeatingQC'] = encoder.fit_transform(df_train['HeatingQC'])
df_train['CentralAir'] = encoder.fit_transform(df_train['CentralAir'])
df_train['Electrical'] = encoder.fit_transform(df_train['Electrical'])
df_train['KitchenQual'] = encoder.fit_transform(df_train['KitchenQual'])
df_train['Functional'] = encoder.fit_transform(df_train['Functional'])
df_train['FireplaceQu'] = encoder.fit_transform(df_train['FireplaceQu'])
df_train['GarageType'] = encoder.fit_transform(df_train['GarageType'])
df_train['GarageFinish'] = encoder.fit_transform(df_train['GarageFinish'])
df_train['GarageQual'] = encoder.fit_transform(df_train['GarageQual'])
df_train['GarageCond'] = encoder.fit_transform(df_train['GarageCond'])
df_train['PavedDrive'] = encoder.fit_transform(df_train['PavedDrive'])
df_train['SaleType'] = encoder.fit_transform(df_train['SaleType'])
df_train['SaleCondition'] = encoder.fit_transform(df_train['SaleCondition'])

In [84]:
x = df_train.iloc[ : , :-1]
y = df_train.iloc[ : , -1]

In [85]:
x

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60.0,3,65.0,8450.0,1,3,3,0,4,0,...,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,8,4
1,20.0,3,80.0,9600.0,1,3,3,0,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,8,4
2,60.0,3,68.0,11250.0,1,0,3,0,4,0,...,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,8,4
3,70.0,3,60.0,9550.0,1,0,3,0,0,0,...,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,8,0
4,60.0,3,84.0,14260.0,1,0,3,0,2,0,...,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,3,62.0,7917.0,1,3,3,0,4,0,...,40.0,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,8,4
1456,20.0,3,85.0,13175.0,1,3,3,0,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2010.0,8,4
1457,70.0,3,66.0,9042.0,1,3,3,0,4,0,...,60.0,0.0,0.0,0.0,0.0,2500.0,5.0,2010.0,8,4
1458,20.0,3,68.0,9717.0,1,3,3,0,4,0,...,0.0,112.0,0.0,0.0,0.0,0.0,4.0,2010.0,8,4


In [86]:
y

,SalePrice
0,208500.0
1,181500.0
2,223500.0
3,140000.0
4,250000.0
...,...
1455,175000.0
1456,210000.0
1457,266500.0
1458,142125.0


In [87]:
scaler = MinMaxScaler()

x = scaler.fit_transform(x)

In [88]:
x

array([[0.23529412, 0.75      , 0.15068493, ..., 0.5       , 1.        ,
        0.8       ],
       [0.        , 0.75      , 0.20205479, ..., 0.25      , 1.        ,
        0.8       ],
       [0.23529412, 0.75      , 0.1609589 , ..., 0.5       , 1.        ,
        0.8       ],
       ...,
       [0.29411765, 0.75      , 0.15410959, ..., 1.        , 1.        ,
        0.8       ],
       [0.        , 0.75      , 0.1609589 , ..., 1.        , 1.        ,
        0.8       ],
       [0.        , 0.75      , 0.18493151, ..., 0.5       , 1.        ,
        0.8       ]])

In [89]:
y

,SalePrice
0,208500.0
1,181500.0
2,223500.0
3,140000.0
4,250000.0
...,...
1455,175000.0
1456,210000.0
1457,266500.0
1458,142125.0


In [91]:
y_df = y.to_frame()

y = scaler.fit_transform(y_df)

In [92]:
y

array([[0.24107763],
       [0.20358284],
       [0.26190807],
       ...,
       [0.321622  ],
       [0.14890293],
       [0.15636717]])

In [93]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42)

In [95]:
model_1 = LinearRegression()
model_2 = SGDRegressor()
model_3 = Lasso()
model_4 = Ridge()
model_5 = ElasticNet()
model_6 = SVR()
model_7 = KNeighborsRegressor()
model_8 = DecisionTreeRegressor()
model_9 = RandomForestRegressor()
model_10 = BaggingRegressor()
model_11 = ExtraTreesRegressor()
model_12 = AdaBoostRegressor()
model_13 = XGBRegressor(verbose=0)
model_15 = LGBMRegressor(verbose=0)

In [96]:
model_1.fit(x_train,y_train)

LinearRegression()

In [97]:
model_2.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor()

In [98]:
model_2.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor()

In [99]:
model_3.fit(x_train,y_train)

Lasso()

In [100]:
model_4.fit(x_train,y_train)

Ridge()

In [101]:
model_4.fit(x_train,y_train)

Ridge()

In [102]:
model_5.fit(x_train,y_train)

ElasticNet()

In [103]:
model_6.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR()

In [104]:
model_7.fit(x_train,y_train)

KNeighborsRegressor()

In [124]:
model_8.fit(x_train,y_train)

DecisionTreeRegressor()

In [125]:
model_9.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [126]:
model_9.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [127]:
model_9.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [128]:
model_12.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor()

In [129]:
model_13.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:02:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [130]:
model_15.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMRegressor(verbose=0)

In [134]:
encoder = LabelEncoder()

df_test['MSZoning'] = encoder.fit_transform(df_test['MSZoning'])
df_test['Street'] = encoder.fit_transform(df_test['Street'])
df_test['LotShape'] = encoder.fit_transform(df_test['LotShape'])
df_test['LandContour'] = encoder.fit_transform(df_test['LandContour'])
df_test['Utilities'] = encoder.fit_transform(df_test['Utilities'])
df_test['LotConfig'] = encoder.fit_transform(df_test['LotConfig'])
df_test['LandSlope'] = encoder.fit_transform(df_test['LandSlope'])
df_test['Neighborhood'] = encoder.fit_transform(df_test['Neighborhood'])
df_test['Condition1'] = encoder.fit_transform(df_test['Condition1'])
df_test['Condition2'] = encoder.fit_transform(df_test['Condition2'])
df_test['BldgType'] = encoder.fit_transform(df_test['BldgType'])
df_test['HouseStyle'] = encoder.fit_transform(df_test['HouseStyle'])
df_test['RoofStyle'] = encoder.fit_transform(df_test['RoofStyle'])
df_test['RoofMatl'] = encoder.fit_transform(df_test['RoofMatl'])
df_test['Exterior1st'] = encoder.fit_transform(df_test['Exterior1st'])
df_test['Exterior2nd'] = encoder.fit_transform(df_test['Exterior2nd'])
df_test['MasVnrType'] = encoder.fit_transform(df_test['MasVnrType'])
df_test['ExterQual'] = encoder.fit_transform(df_test['ExterQual'])
df_test['ExterCond'] = encoder.fit_transform(df_test['ExterCond'])
df_test['Foundation'] = encoder.fit_transform(df_test['Foundation'])
df_test['BsmtQual'] = encoder.fit_transform(df_test['BsmtQual'])
df_test['BsmtCond'] = encoder.fit_transform(df_test['BsmtCond'])
df_test['BsmtExposure'] = encoder.fit_transform(df_test['BsmtExposure'])
df_test['BsmtFinType1'] = encoder.fit_transform(df_test['BsmtFinType1'])
df_test['BsmtFinType2'] = encoder.fit_transform(df_test['BsmtFinType2'])
df_test['Heating'] = encoder.fit_transform(df_test['Heating'])
df_test['HeatingQC'] = encoder.fit_transform(df_test['HeatingQC'])
df_test['CentralAir'] = encoder.fit_transform(df_test['CentralAir'])
df_test['Electrical'] = encoder.fit_transform(df_test['Electrical'])
df_test['KitchenQual'] = encoder.fit_transform(df_test['KitchenQual'])
df_test['Functional'] = encoder.fit_transform(df_test['Functional'])
df_test['FireplaceQu'] = encoder.fit_transform(df_test['FireplaceQu'])
df_test['GarageType'] = encoder.fit_transform(df_test['GarageType'])
df_test['GarageFinish'] = encoder.fit_transform(df_test['GarageFinish'])
df_test['GarageQual'] = encoder.fit_transform(df_test['GarageQual'])
df_test['GarageCond'] = encoder.fit_transform(df_test['GarageCond'])
df_test['PavedDrive'] = encoder.fit_transform(df_test['PavedDrive'])
df_test['SaleType'] = encoder.fit_transform(df_test['SaleType'])
df_test['SaleCondition'] = encoder.fit_transform(df_test['SaleCondition'])

In [135]:
df_test.head(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20.0,2,80.0,11622.0,1,3,3,0,4,0,...,0.0,0.0,0.0,120.0,0.0,0.0,6.0,2010.0,8,4
1,20.0,3,81.0,14267.0,1,0,3,0,0,0,...,36.0,0.0,0.0,0.0,0.0,12500.0,6.0,2010.0,8,4
2,60.0,3,74.0,13830.0,1,0,3,0,4,0,...,34.0,0.0,0.0,0.0,0.0,0.0,3.0,2010.0,8,4
3,60.0,3,78.0,9978.0,1,0,3,0,4,0,...,36.0,0.0,0.0,0.0,0.0,0.0,6.0,2010.0,8,4
4,120.0,3,43.0,5005.0,1,0,1,0,4,0,...,82.0,0.0,0.0,144.0,0.0,0.0,1.0,2010.0,8,4
5,60.0,3,75.0,10000.0,1,0,3,0,0,0,...,84.0,0.0,0.0,0.0,0.0,0.0,4.0,2010.0,8,4
6,20.0,3,67.0,7980.0,1,0,3,0,4,0,...,21.0,0.0,0.0,0.0,0.0,500.0,3.0,2010.0,8,4
7,60.0,3,63.0,8402.0,1,0,3,0,4,0,...,75.0,0.0,0.0,0.0,0.0,0.0,5.0,2010.0,8,4
8,20.0,3,85.0,10176.0,1,3,3,0,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2010.0,8,4
9,20.0,3,70.0,8400.0,1,3,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2010.0,8,4


In [136]:
y_pred = model_13.predict(df_test)
y_pred

array([0.6157213 , 0.5433999 , 0.81146735, ..., 0.65946186, 0.5150808 ,
       0.7655165 ], dtype=float32)

In [137]:
y_pred_reshaped = y_pred.reshape(-1, 1)

scaled_data = scaler.transform(y_pred_reshaped)


y_pred_rescaler = scaler.inverse_transform(y_pred_reshaped)


y_pred_rescaler

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


array([[478280.9 ],
       [426202.25],
       [619237.6 ],
       ...,
       [509778.47],
       [405809.7 ],
       [586148.44]], dtype=float32)

In [142]:
submission = pd.read_csv('./drive/MyDrive/Datasets/sample_submission.csv')

In [143]:
submission['SalePrice']= y_pred_rescaler

In [144]:
submission.to_csv('submission.csv', index=False)